### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [46]:
import json
import music21
from fastai.text import *
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio

In [3]:
path = Path('../data/midi')

In [4]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('../data/midi/cprato_midi'),
 PosixPath('../data/midi/Wikifonia'),
 PosixPath('../data/midi/transposed'),
 PosixPath('../data/midi/hymn'),
 PosixPath('../data/midi/classic_piano_scrape'),
 PosixPath('../data/midi/touhou_scrape'),
 PosixPath('../data/midi/hooktheory'),
 PosixPath('../data/midi/ecomp_midi'),
 PosixPath('../data/midi/freemidi_scrape'),
 PosixPath('../data/midi/piano_e_competition'),
 PosixPath('../data/midi/midiworld_scrape')]

## General stream parsing

In [6]:
def get_music21_attr(fp, transpose=True):
    stream = music21.converter.parse(fp)
    stream_attr = get_stream_attr(stream)
    if transpose:
        new_stream, new_stream_attr = convert2keyc(fp, stream)
    return {**stream_attr, **new_stream_attr}

In [35]:
# converting everything into the key of C major or A minor
# https://gist.github.com/aldous-rey/68c6c43450517aa47474
# major conversions
# majors = dict([("A-", 4),("A", 3),("B-", 2) ,("B", 1),("C-", 1),("C", 0) ,("D-", -1),("C#", -1),("D", -2),          ("E-", -3),("E", -4),("F", -5),("G-", 6),("F#", 6),("G", 5)])
# minors = dict([("A-", 1),("A", 0),("B-", -1),("B", -2),         ("C", -3),("D-", -4),           ("D", -5),("D#", 6),("E-", 6), ("E", 5), ("F", 4), ("G-", 3),          ("G", 2)])

In [10]:
majors = {
    'A-': 4,
    'A':  3,
    'B-': 2,
    'B':  1,'C-': 1,
    'C':  0,
    'D-':-1,'C#':-1,
    'D': -2,
    'E-':-3,
    'E': -4,
    'F': -5,
    'G-': 6,
    'F#': 6,
    'G':  5
}

minors = {
    'A-': 1,'G#': 1,
    'A':  0,
    'B-':-1,
    'B': -2,
    'C': -3,
    'D-':-4,'C#':-4,
    'D': -5,
    'E-': 6,'D#': 6,
    'E':  5,
    'F':  4,
    'G-': 3,'F#': 3,
    'G':  2
}

def 

def convert2keyc(file, score=None, out_file=None):
    transposed_file = Path(str(file).replace('data/midi/', 'data/midi/transposed/'))
    if not transposed_file.exists():
        if not score:
            score = music21.converter.parse(file)
        key = score.analyze('key')

        if key.mode == "major":   halfSteps = majors[key.tonic.name]
        elif key.mode == "minor": halfSteps = minors[key.tonic.name]

        transposed_score = score.transpose(halfSteps)
        Path(transposed_file).parent.mkdir(parents=True, exist_ok=True)
        transposed_score.write('midi',transposed_file)
    else:
        transposed_score = music21.converter.parse(transposed_file)
    newkey = transposed_score.analyze('key')
    metadata = {
        'inferred_keyc': f'{newkey.tonic.name} {newkey.mode}',
        'midi_keyc': str(transposed_file),
    }
    return transposed_score, metadata

In [37]:
def get_stream_attr(s):
    instruments = [i.instrumentName for i in list(s.getInstruments(recurse=True)) if i.instrumentName]
    metronome = list(filter(lambda x: isinstance(x, music21.tempo.MetronomeMark), s.flat))[0]
    bpm = metronome.getQuarterBPM()
    key = s.analyze('key')
    s_flat = s.flat
    time_sig = s_flat.timeSignature.ratioString if hasattr(s_flat.timeSignature, 'ratioString') else None
    return {
        'instruments': instruments,
        'bpm': bpm,
        'inferred_key': f'{key.tonic.name} {key.mode}',
        'seconds': s_flat.seconds,
        'time_signature': s_flat.timeSignature.ratioString,
    }

### Gather Hooktheory Data

In [12]:
hook_path = Path('../data/midi/hooktheory')

In [13]:
song_info = list((hook_path/'xml').glob('*/*/*/*.json'))
def song_key(s): return '_'.join(s.parts[-3:-1])
key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
len(song_info)

11873

In [14]:
# song_xml = list((hook_path/'xml').glob('*/*/*/*.xml')); len(song_xml)
song_json = list((hook_path/'event').glob('*/*/*/*_key.json')); len(song_json)
# song_mid = list((hook_path/'pianoroll').glob('*/*/*/*_nokey.mid')); len(song_mid)

19876

In [15]:
def get_hooktheory_attr(song_json_path, key2info):
    song_json = json.load(open(song_json_path, 'r'))
    metadata = song_json['metadata']
    song_info = key2info[song_key(song_json_path)].copy()
    artist = song_json_path.parts[-3]
    section = song_json_path.name.split('_')[0]
    midi_path = str(song_json_path.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')
    
    # convert stream here
    metadata = {
        'artist': artist,
        'section': section,
        'original_path': midi_path,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'title': metadata['title'],
        'mode': metadata['mode']
    }
    try:
        stream_info = get_music21_attr(midi_path)
    except Exception as e:
        stream_info = {}
        print('Could not parse stream', midi_path, e)
    return {**metadata, **stream_info}

In [16]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000], key2info); hook_out

In [18]:
hooktheory_list = [get_hooktheory_attr(s, key2info) for s in tqdm(song_json, total=len(song_json))]

  0%|          | 62/19876 [00:06<36:51,  8.96it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key.mid list index out of range


  1%|          | 221/19876 [00:51<1:41:05,  3.24it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key.mid list index out of range


  3%|▎         | 659/19876 [02:55<1:36:34,  3.32it/s]

...

 99%|█████████▉| 19669/19876 [1:30:45<01:09,  2.97it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/a/antonim/melancholy-soldier/bridge_key.mid list index out of range


 99%|█████████▉| 19731/19876 [1:31:03<00:45,  3.16it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/a/asf/asdfas/intro-and-verse_key.mid list index out of range


100%|██████████| 19876/19876 [1:31:45<00:00,  3.60it/s]


Save song_list

In [19]:
song_list_path = 'song_list_metadata.json'

In [20]:
json.dump(hooktheory_list, open(song_list_path, 'w'))

In [21]:
hooktheory_list[-3:]

[{'artist': 'apink',
  'section': 'chorus',
  'original_path': '../data/midi/hooktheory/pianoroll/a/apink/only-one/chorus_key.mid',
  'parts': ['chorus'],
  'song_url': 'https://www.hooktheory.com/theorytab/view/apink/only-one',
  'genres': ['K-pop'],
  'title': 'Only One',
  'mode': '1',
  'instruments': ['Piano', 'Piano'],
  'bpm': 102.0,
  'inferred_key': 'F# major',
  'seconds': 18.823529411764707,
  'time_signature': '4/4',
  'inferred_keyc': 'C major',
  'midi_keyc': '../data/midi/transposed/hooktheory/pianoroll/a/apink/only-one/chorus_key.mid'},
 {'artist': 'apink',
  'section': 'chorus',
  'original_path': '../data/midi/hooktheory/pianoroll/a/apink/promise-u/chorus_key.mid',
  'parts': ['chorus'],
  'song_url': 'https://www.hooktheory.com/theorytab/view/apink/promise-u',
  'genres': ['K-pop'],
  'title': 'pu',
  'mode': '1',
  'instruments': ['Piano', 'Piano'],
  'bpm': 70.0,
  'inferred_key': 'A- major',
  'seconds': 27.428571428571427,
  'time_signature': '4/4',
  'inferred_k

In [22]:
hooktheory_list_loaded = json.load(open(song_list_path, 'r'))

In [23]:
hooktheory_list_loaded[100]

{'artist': 'weezer',
 'section': 'bridge',
 'original_path': '../data/midi/hooktheory/pianoroll/w/weezer/say-it-aint-so/bridge_key.mid',
 'parts': ['verse', 'chorus', 'bridge'],
 'song_url': 'https://www.hooktheory.com/theorytab/view/weezer/say-it-aint-so',
 'genres': ['Pop', 'Rock'],
 'title': "Say It Ain't So",
 'mode': '1',
 'instruments': ['Piano', 'Piano'],
 'bpm': 70.0,
 'inferred_key': 'B- major',
 'seconds': 27.428571428571427,
 'time_signature': '4/4',
 'inferred_keyc': 'C major',
 'midi_keyc': '../data/midi/transposed/hooktheory/pianoroll/w/weezer/say-it-aint-so/bridge_key.mid'}

### Gather FreeMidi

In [25]:
freemidi_path = Path('../data/midi/freemidi_scrape'); freemidi_path.ls()

[PosixPath('../data/midi/freemidi_scrape/genre-disco'),
 PosixPath('../data/midi/freemidi_scrape/genre-pop'),
 PosixPath('../data/midi/freemidi_scrape/genre-dance-eletric'),
 PosixPath('../data/midi/freemidi_scrape/genre-punk'),
 PosixPath('../data/midi/freemidi_scrape/genre-hip-hop-rap'),
 PosixPath('../data/midi/freemidi_scrape/genre-rock')]

In [26]:
def parse_freemidi_songs(fp, genre=None):
    try:
        name = fp.with_suffix('').name.split(' - ')
        artist = name[0]
        title = name[-1]
        attr = get_music21_attr(fp)
    except Exception as e:
        print('Exeption:', fp, e)
    
    metadata = {
        'artist': artist,
        'title': title,
        'midi': str(fp),
        'genre': genre
    }
    return {**metadata, **attr}

In [35]:
def parse_freemidi_genre(genre_path, genre):
    file_list = genre_path.ls()
    parsed_songs = [parse_freemidi_songs(fp, genre) for fp in tqdm(file_list, total=len(file_list))]
    
    metadata_path = f'freemidi_{genre}_metadata.json'
    json.dump(parsed_songs, open(genre_path.parent/metadata_path, 'w'))
    
    return parsed_songs, metadata_path

In [ ]:
# sanity check
freemidi_out = parse_freemidi(dance_midi_files[0], genre='dance')

In [38]:
dance_files = (freemidi_path/'genre-dance-eletric/').ls()

In [39]:
atb = dance_files[1]

In [59]:
atb

PosixPath('../data/midi/freemidi_scrape/genre-dance-eletric/ATB - Dont stop.mid')

In [41]:
old_song = stream = music21.converter.parse(atb)

In [42]:
new_song = convert2keyc(atb)

In [56]:
music21.environment.get("musicxmlPath")

PosixPath('/usr/bin/musescore')

In [58]:
old_song.show('musicxml')

In [ ]:
music21.environment.set("musicxmlPath", "/usr/bin/musescore")
music21.environment.set("graphicsPath", "/usr/bin/musescore")

In [43]:
old_song.show('midi')

In [54]:
list(old_song.flat)

[<music21.instrument.ElectricBass Electric Bass>,
 <music21.instrument.Instrument >,
 <music21.instrument.Instrument >,
 <music21.instrument.Instrument >,
 <music21.instrument.ElectricGuitar Electric Guitar>,
 <music21.instrument.Whistle Whistle>,
 <music21.instrument.ElectricOrgan Electric Organ>,
 <music21.instrument.Glockenspiel Glockenspiel>,
 <music21.instrument.Instrument >,
 <music21.instrument.ElectricGuitar Electric Guitar>,
 <music21.tempo.MetronomeMark allegro Quarter=133.0>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.key.Key of C major>,
 <music21.meter.TimeSignature 4/4>,
 <music21.meter.TimeSignature 4/4>,
 <music21.meter.TimeSignature 4/4>,
 <music21.meter.TimeSignature 4/4>,
 <music21.meter.TimeSignature 4/4>,
 <music21.meter.Ti

In [50]:
Image(filename=str(old_song.write('lily.png')))

LilyTranslateException: DurationException for durationObject <music21.duration.Duration 68.5>: Could not determine durationNumber from inexpressible

In [44]:
new_song.show('midi')

AttributeError: 'tuple' object has no attribute 'show'

In [36]:
freemidi_dance_list, freemidi_dance_path = parse_freemidi_genre(freemidi_path/'genre-dance-eletric', 'dance')


  0%|          | 4/1122 [00:57<3:55:59, 12.67s/it]

Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Tune Up - Bounce.mid 'NoneType' object has no attribute 'ratioString'


UnboundLocalError: local variable 'attr' referenced before assignment

In [ ]:
freemidi_dance_list, freemidi_dance_path = parse_freemidi_genre(freemidi_path/'genre-pop', 'pop')

In [ ]:
freemidi_dance_list = json.load(open(metadata_path, 'r'))

### Gather Cprato

### Yamaha - piano

In [ ]:
ecomp_path = Path('../data/midi/piano_e_competition')
ecomp_scrape_path = Path('../data/midi/ecomp_midi')

### Classic Piano

In [ ]:
classic_scrape_path = Path('../data/midi/classic_piano_scrape')

### Creating CSV

In [ ]:
import csv
toCSV = [{'name':'bob','age':25,'weight':200},
         {'name':'jim','age':31,'weight':180}]
keys = toCSV[0].keys()
with open('people.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)

## Preparing the data

In [3]:
class MusicTokenizer():
    def __init__(self):
        super().__init__()
        self.n_cpus = num_cpus()
        
    def process_text(self, t:str) -> List[str]:
        return t.split(" ")
    
    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        return [self.process_text(t) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])


In [4]:
path = Path('data/composers/notewise/piano_solo/note_range62/sample_freq12')
path.ls()[:5]

[PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/jazz'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/ravel'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/bach'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/schumann'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/liszt')]

In [5]:
bs=32

In [6]:
if (path/'tmp/itos.pkl').exists():
# if False:
    data = TextLMDataBunch.load(path, bs=bs)
else:
    p = [OpenFileProcessor(), TokenizeProcessor(tokenizer=MusicTokenizer(), chunksize=10), NumericalizeProcessor(vocab=None, max_vocab=500)]

    data = (TextList.from_folder(path, recurse=True, processor=p)
            .random_split_by_pct(0.05, seed=6)
            .label_for_lm()
            .databunch(bs=bs))
    data.save('tmp')
vocab = data.train_ds.vocab
len(data.train_ds), len(data.valid_ds), len(data.train_ds.vocab.itos)

(1797, 94, 158)

In [7]:
t = data.train_ds[0][0]
t.text[:50], t.data

('xxbos wait25 wait25 wait25 wait25 wait25 wait25 wa',
 array([  2, 124, 124, 124, ...,   9, 105,   9, 157]))

In [8]:
data.show_batch()

idx,text
0,xxbos p21 p28 p36 wait7 endp36 wait2 p29 p36 wait6 endp28 wait25 endp29 endp36 wait1 endp21 wait1 p28 wait2 p27 endp28 wait2 endp27 wait2 p20 p28 wait18 p29 p35 wait14 endp28 wait6 endp29 endp35 wait2 p28 wait5 endp28 wait2 endp20 wait1 p19 p28 wait7 endp28 wait4 endp19 wait7 p40 p47 wait2 p35 p36 p43 wait2 endp43 wait9 endp35 wait2 endp40 endp47 wait2 endp36 wait5 p33 p36 p40 p45 wait2 endp33
1,p24 wait8 endp24 wait4 p24 wait5 endp24 wait1 p20 wait3 endp20 wait3 p20 wait1 endp34 wait1 endp25 endp30 wait4 p36 wait2 endp20 endp36 wait4 p13 p20 p24 endp24 p28 p31 p36 wait2 endp28 endp31 endp36 wait1 endp20 wait3 p22 p27 p32 p37 wait2 endp32 endp37 wait1 endp22 endp27 wait3 p13 p23 p28 p32 wait2 endp23 wait4 p23 endp23 wait1 endp13 wait1 endp28 wait4 p6 p22 p28 wait6 p13 wait1 endp22
2,wait1 endp27 wait2 endp34 wait1 p39 p43 wait3 endp39 endp43 wait2 p22 p27 p31 wait3 endp22 endp27 endp31 wait1 endp19 wait2 p34 wait2 endp34 wait2 endp46 wait2 p39 endp39 p43 endp43 wait9 p14 p38 wait6 p26 wait1 endp14 wait2 endp26 wait1 p33 wait1 endp38 wait3 p7 p19 p39 wait1 endp33 wait2 endp7 wait1 p27 wait1 endp19 wait2 endp27 wait3 p34 wait3 endp34 p43 wait6 p27 wait1 endp43 wait1 endp27 wait1
3,p4 p35 wait3 endp35 wait1 p32 wait2 endp16 wait1 endp4 p9 endp32 p33 wait6 p12 wait1 endp9 wait6 endp12 wait2 p16 wait6 p21 wait1 endp16 wait5 endp33 wait1 endp21 wait1 p9 p16 p40 wait7 endp40 p45 wait6 p9 p21 wait1 endp16 wait4 endp45 wait1 p36 wait3 p35 endp36 wait3 p33 endp35 wait2 endp9 endp21 wait1 p20 p23 endp33 p35 wait4 p44 wait1 endp35 wait2 endp44 wait1 p47 wait7 p16 p20
4,endp52 endp56 wait6 p44 p47 p52 p56 wait1 p23 p28 p32 p35 wait3 endp35 wait2 endp28 endp32 wait2 endp47 endp56 wait1 endp23 wait2 endp44 endp52 wait14 p4 wait2 endp4 wait10 p52 wait3 p24 p28 p31 p36 p40 wait2 endp24 endp28 endp31 endp36 endp40 endp52 wait6 p24 endp24 p28 p31 p36 p40 p50 wait2 endp28 endp31 endp36 wait1 endp40 endp50 wait4 p24 endp24 p28 p31 p36 p40 p48 wait2 endp28 endp31
